<img src="dataset_5-cover.jpg" alt="Circular Image" style="border-radius: 50%; display: block; margin: 0 auto; width: 200px; height: 200px;">
<h1>Superstore Marketing Campaign Dataset</h1>
<p><i>Sample customer data for analysis of a targeted Membership Offer</i></p>
<h2>About Dataset</h2>
<p><b>Context</b>: A superstore is planning for the year-end sale. They want to launch a new offer - gold membership, that gives a 20% discount on all purchases, for only $499 which is $999 on other days. It will be valid only for existing customers and the campaign through phone calls is currently being planned for them. The management feels that the best way to reduce the cost of the campaign is to make a predictive model which will classify customers who might purchase the offer.</p>
<h2>Data Source:</h2>
<p><b>Kaggle: </b> <a href="https://www.kaggle.com/datasets/ahsan81/superstore-marketing-campaign-dataset">Click Here!!!</a></p>

In [6]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

con = duckdb.connect("data/db/superstore.duckdb")

In [7]:
query = """
SHOW TABLES;
"""
con.sql(query).df()

,name
0,campaign_response
1,customer_behavior
2,customer_purchases
3,customers


# 1. Data Profiling Queries

<p>The process for detecting issues (missing data, outliers, inconsistencies) before data is used.</p>


## 1. Size and Duplicates

### 1. Duplicates

In [11]:
query= """
SELECT 'customers' AS table_name, COUNT(*) AS total_rows,
COUNT(DISTINCT Id) AS unique_ids
FROM customers
UNION ALL SELECT 'campaign_response', COUNT(*), COUNT(DISTINCT Id) FROM campaign_response
UNION ALL SELECT 'customer_behavior', COUNT(*), COUNT(DISTINCT Id) FROM customer_behavior
UNION ALL SELECT 'customer_purchases', COUNT(*), COUNT(DISTINCT Id) FROM customer_purchases;"""
con.sql(query).df()

,table_name,total_rows,unique_ids
0,customers,2240,2240
1,campaign_response,2240,2240
2,customer_behavior,2240,2240
3,customer_purchases,2240,2240


### 2. Missing Values